In [ ]:
# ЯЧЕЙКА 6: Выполнение реальных задач агентами с RAG
import asyncio
import time

rag_agents = globals()['RAG_AGENTS']

print("🚀 Демонстрация выполнения реальных задач агентами")
print("=" * 60)

# Задачи для демонстрации реальной работы системы
real_tasks = [
    {
        "agent_id": "lead_qualification",
        "task_data": {
            "client_name": "TechMarket Pro",
            "domain": "techmarket-pro.ru", 
            "monthly_budget": 500000,
            "business_type": "E-commerce",
            "contact_person": "Иван Петров",
            "email": "ivan@techmarket-pro.ru",
            "phone": "+7 (495) 123-45-67",
            "goals": "Увеличить органический трафик на 200%",
            "timeline": "6 месяцев",
            "competitors": ["wildberries.ru", "market.yandex.ru"]
        }
    },
    {
        "agent_id": "technical_seo_auditor",
        "task_data": {
            "domain": "techmarket-pro.ru",
            "audit_type": "comprehensive",
            "focus_areas": ["page_speed", "mobile", "crawling", "indexing"],
            "depth": "detailed",
            "include_competitive": True
        }
    },
    {
        "agent_id": "content_strategy", 
        "task_data": {
            "client_domain": "techmarket-pro.ru",
            "industry": "Electronics E-commerce",
            "target_audience": "IT-специалисты, геймеры",
            "content_goals": ["increase_traffic", "improve_conversion", "brand_awareness"],
            "budget": 200000,
            "timeline": "3 months"
        }
    }
]

task_results = {}

for task in real_tasks:
    agent_id = task["agent_id"]
    task_data = task["task_data"]
    
    if agent_id in rag_agents:
        print(f"\n🤖 АГЕНТ: {agent_id}")
        print(f"📋 ЗАДАЧА: {list(task_data.keys())[:3]}...")
        print("-" * 50)
        
        try:
            start_time = time.time()
            
            # Выполняем задачу с помощью реального агента
            result = await rag_agents[agent_id].process_task_with_retry(task_data)
            
            execution_time = time.time() - start_time
            
            print(f"✅ Статус: {result.get('status', result.get('success', 'unknown'))}")
            print(f"⏱️ Время выполнения: {execution_time:.2f}с")
            print(f"📊 Токенов использовано: {result.get('tokens_used', {}).get('total_tokens', 'N/A')}")
            
            # Показываем краткий результат
            if result.get('success', False):
                result_preview = str(result.get('result', ''))[:200]
                print(f"📝 Результат: {result_preview}...")
            else:
                error = result.get('error', 'Unknown error')
                print(f"❌ Ошибка: {error[:100]}...")
            
            task_results[agent_id] = {
                'success': result.get('success', False),
                'execution_time': execution_time,
                'tokens_used': result.get('tokens_used', {}),
                'has_knowledge_context': result.get('knowledge_context_length', 0) > 0,
                'agent_level': rag_agents[agent_id].agent_level
            }
            
        except Exception as e:
            print(f"❌ Критическая ошибка: {str(e)}")
            task_results[agent_id] = {
                'success': False,
                'error': str(e),
                'execution_time': time.time() - start_time
            }
    else:
        print(f"⚠️ Агент {agent_id} не найден")

# Итоговая статистика выполнения задач
print("\n" + "=" * 60)
print("📊 СТАТИСТИКА ВЫПОЛНЕНИЯ ЗАДАЧ:")
print("=" * 60)

successful_tasks = len([r for r in task_results.values() if r.get('success', False)])
total_execution_time = sum(r.get('execution_time', 0) for r in task_results.values())
avg_execution_time = total_execution_time / max(1, len(task_results))

print(f"✅ Успешно выполнено: {successful_tasks}/{len(real_tasks)} задач")
print(f"⏱️ Общее время выполнения: {total_execution_time:.1f}с")
print(f"📊 Среднее время на задачу: {avg_execution_time:.1f}с")

# Анализ по уровням агентов
levels_stats = {}
for agent_id, result in task_results.items():
    level = result.get('agent_level', 'unknown')
    if level not in levels_stats:
        levels_stats[level] = {'total': 0, 'successful': 0}
    levels_stats[level]['total'] += 1
    if result.get('success', False):
        levels_stats[level]['successful'] += 1

print(f"\n📈 СТАТИСТИКА ПО УРОВНЯМ АГЕНТОВ:")
for level, stats in levels_stats.items():
    success_rate = (stats['successful'] / stats['total']) * 100 if stats['total'] > 0 else 0
    print(f"   {level:12}: {stats['successful']}/{stats['total']} ({success_rate:.0f}%)")

# Сохраняем результаты
globals()['TASK_RESULTS'] = task_results

print(f"\n🎯 Реальная система работает: {successful_tasks >= 2}")

In [ ]:
# ЯЧЕЙКА 1: Установка зависимостей и настройка окружения\nimport os\nimport sys\nimport subprocess\n\n# Клонируем проект если нужно\nif not os.path.exists('/content/ai-seo-architects'):\n    subprocess.run(['git', 'clone', 'https://github.com/Andrew821667/ai-seo-architects.git', '/content/ai-seo-architects'])\n    print('✅ Проект клонирован')\n\n# Переходим в директорию проекта\nos.chdir('/content/ai-seo-architects')\nsys.path.append('/content/ai-seo-architects')\n\n# Устанавливаем зависимости\ndependencies = [\n    'openai==1.3.7',\n    'langchain==0.0.348', \n    'langchain-openai==0.0.2',\n    'langgraph==0.0.20',\n    'faiss-cpu==1.7.4',\n    'numpy==1.24.3',\n    'nest-asyncio==1.5.8'\n]\n\nfor dep in dependencies:\n    subprocess.run(['pip', 'install', '-q', dep])\n    print(f'✅ {dep}')\n\nimport asyncio\nimport nest_asyncio\nnest_asyncio.apply()\n\nprint('🎯 Окружение настроено')

In [ ]:
# ЯЧЕЙКА 2: Настройка OpenAI API ключа\nimport os\nfrom google.colab import userdata\nimport getpass\n\ntry:\n    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')\n    if OPENAI_API_KEY:\n        os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY\n        print('✅ OpenAI API ключ загружен из Colab secrets')\n    else:\n        raise Exception('API ключ не найден в secrets')\nexcept:\n    OPENAI_API_KEY = getpass.getpass('Введите OpenAI API ключ: ')\n    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY\n    print('✅ API ключ установлен')\n\n# Тестируем подключение\nimport openai\nclient = openai.OpenAI(api_key=OPENAI_API_KEY)\nresponse = client.chat.completions.create(\n    model='gpt-3.5-turbo',\n    messages=[{'role': 'user', 'content': 'Тест'}],\n    max_tokens=10\n)\nprint(f'✅ OpenAI API работает: {response.choices[0].message.content}')

In [ ]:
# ЯЧЕЙКА 3: Инициализация Knowledge Manager (РЕАЛЬНЫЙ КОД ПРОЕКТА)\nfrom knowledge.knowledge_manager import knowledge_manager\nfrom core.config import config\nimport os\n\nprint('🔧 Инициализация Knowledge Manager')\nprint(f'📁 База знаний: {config.KNOWLEDGE_BASE_PATH}')\nprint(f'🔍 RAG включен: {config.ENABLE_RAG}')\nprint(f'📊 Chunk size: {config.RAG_CHUNK_SIZE}')\nprint(f'🎯 Top K: {config.RAG_TOP_K}')\n\n# Проверяем структуру директорий знаний\nknowledge_path = '/content/ai-seo-architects/knowledge'\nif os.path.exists(knowledge_path):\n    for level in ['executive', 'management', 'operational']:\n        level_path = os.path.join(knowledge_path, level)\n        if os.path.exists(level_path):\n            md_files = [f for f in os.listdir(level_path) if f.endswith('.md')]\n            print(f'📚 {level}: {len(md_files)} файлов знаний')\n        else:\n            print(f'❌ Директория {level} не найдена')\nelse:\n    print(f'❌ Директория знаний не найдена: {knowledge_path}')\n\n# Инициализируем базы знаний для всех агентов\nprint('\n🚀 Инициализация всех баз знаний...')\ninitialization_results = knowledge_manager.initialize_all_agents_knowledge()\n\nsuccessful_agents = [agent for agent, success in initialization_results.items() if success]\nfailed_agents = [agent for agent, success in initialization_results.items() if not success]\n\nprint(f'\n✅ Успешно инициализировано: {len(successful_agents)} агентов')\nprint(f'❌ Ошибки инициализации: {len(failed_agents)} агентов')\n\nif successful_agents:\n    print(f'\n✅ Успешные агенты: {", ".join(successful_agents[:5])}{"..." if len(successful_agents) > 5 else ""}')\n    \nif failed_agents:\n    print(f'❌ Проблемные агенты: {", ".join(failed_agents)}')\n\n# Сохраняем результаты для следующих ячеек\nglobals()['KNOWLEDGE_MANAGER'] = knowledge_manager\nglobals()['SUCCESSFUL_AGENTS'] = successful_agents\nglobals()['INITIALIZATION_RESULTS'] = initialization_results

In [ ]:
# ЯЧЕЙКА 4: Импорт и инициализация агентов (РЕАЛЬНЫЕ КЛАССЫ ПРОЕКТА)\nfrom agents.executive.chief_seo_strategist import ChiefSEOStrategistAgent\nfrom agents.executive.business_development_director import BusinessDevelopmentDirectorAgent\nfrom agents.management.task_coordination import TaskCoordinationAgent\nfrom agents.management.sales_operations_manager import SalesOperationsManagerAgent\nfrom agents.management.technical_seo_operations_manager import TechnicalSEOOperationsManagerAgent\nfrom agents.management.client_success_manager import ClientSuccessManagerAgent\nfrom agents.operational.lead_qualification import LeadQualificationAgent\nfrom agents.operational.sales_conversation import SalesConversationAgent\nfrom agents.operational.proposal_generation import ProposalGenerationAgent\nfrom agents.operational.technical_seo_auditor import TechnicalSEOAuditorAgent\nfrom agents.operational.content_strategy import ContentStrategyAgent\nfrom agents.operational.link_building import LinkBuildingAgent\nfrom agents.operational.competitive_analysis import CompetitiveAnalysisAgent\nfrom agents.operational.reporting import ReportingAgent\n\nprint('🤖 Инициализация всех 14 агентов с RAG')\n\n# Конфигурация агентов\nagents_config = [\n    ('chief_seo_strategist', ChiefSEOStrategistAgent, 'executive'),\n    ('business_development_director', BusinessDevelopmentDirectorAgent, 'executive'),\n    ('task_coordination', TaskCoordinationAgent, 'management'),\n    ('sales_operations_manager', SalesOperationsManagerAgent, 'management'),\n    ('technical_seo_operations_manager', TechnicalSEOOperationsManagerAgent, 'management'),\n    ('client_success_manager', ClientSuccessManagerAgent, 'management'),\n    ('lead_qualification', LeadQualificationAgent, 'operational'),\n    ('sales_conversation', SalesConversationAgent, 'operational'),\n    ('proposal_generation', ProposalGenerationAgent, 'operational'),\n    ('technical_seo_auditor', TechnicalSEOAuditorAgent, 'operational'),\n    ('content_strategy', ContentStrategyAgent, 'operational'),\n    ('link_building', LinkBuildingAgent, 'operational'),\n    ('competitive_analysis', CompetitiveAnalysisAgent, 'operational'),\n    ('reporting', ReportingAgent, 'operational')\n]\n\n# Инициализируем всех агентов\nrag_agents = {}\nsuccessful_inits = 0\n\nfor agent_id, AgentClass, level in agents_config:\n    try:\n        # Создаем агента с RAG поддержкой\n        agent = AgentClass(\n            agent_id=agent_id,\n            name=agent_id.replace('_', ' ').title(),\n            agent_level=level,\n            rag_enabled=True,\n            knowledge_base=f'{level}/{agent_id}.md'\n        )\n        \n        rag_agents[agent_id] = agent\n        successful_inits += 1\n        \n        print(f'✅ {agent_id:35} | {level:12} | RAG: {agent.rag_enabled}')\n        \n    except Exception as e:\n        print(f'❌ {agent_id:35} | Ошибка: {str(e)[:40]}')\n\nprint(f'\n📊 Результат инициализации: {successful_inits}/14 агентов')\n\n# Тест RAG системы с реальным агентом\nif 'lead_qualification' in rag_agents:\n    test_agent = rag_agents['lead_qualification']\n    test_query = 'Как квалифицировать лида с бюджетом 100000 рублей?'\n    \n    # Получаем контекст знаний напрямую\n    context = await test_agent.get_knowledge_context(test_query, k=2)\n    print(f'\n🧪 Тест RAG поиска:')\n    print(f'📝 Запрос: {test_query}')\n    print(f'📚 Контекст ({len(context)} символов): {context[:200] if context else "Контекст не найден"}...')\n\n# Сохраняем для следующих ячеек\nglobals()['RAG_AGENTS'] = rag_agents\nglobals()['SUCCESSFUL_AGENTS_COUNT'] = successful_inits\n\nprint(f'\n🎯 Система готова: {successful_inits >= 10}')

In [ ]:
# ЯЧЕЙКА 5: Демонстрация RAG поиска по всем агентам\nimport asyncio\nimport time\n\nknowledge_manager = globals()['KNOWLEDGE_MANAGER']\nrag_agents = globals()['RAG_AGENTS']\n\nprint('🔍 Демонстрация RAG поиска по всем агентам')\nprint('=' * 60)\n\n# Тестовые запросы для разных типов агентов\ntest_queries = [\n    ('lead_qualification', 'Как квалифицировать лида с бюджетом 500000 рублей?'),\n    ('technical_seo_auditor', 'Проведи технический аудит сайта для e-commerce'),\n    ('content_strategy', 'Создай контентную стратегию для B2B компании'),\n    ('proposal_generation', 'Создай коммерческое предложение для крупного клиента'),\n    ('competitive_analysis', 'Проанализируй конкурентов в нише SEO услуг'),\n    ('chief_seo_strategist', 'Разработай SEO стратегию для международной экспансии')\n]\n\nrag_demo_results = {}\n\nfor agent_id, query in test_queries:\n    if agent_id in rag_agents:\n        print(f'\n🤖 АГЕНТ: {agent_id}')\n        print(f'❓ ЗАПРОС: {query}')\n        print('-' * 50)\n        \n        try:\n            # Поиск через Knowledge Manager\n            start_time = time.time()\n            relevant_docs = knowledge_manager.search_knowledge(agent_id, query, k=3)\n            search_time = time.time() - start_time\n            \n            if relevant_docs:\n                total_chars = sum(len(doc.page_content) for doc in relevant_docs)\n                print(f'✅ Найдено: {len(relevant_docs)} документов ({total_chars} символов)')\n                print(f'⏱️ Время поиска: {search_time:.2f}с')\n                \n                # Показываем первый результат\n                first_doc = relevant_docs[0]\n                source = first_doc.metadata.get('source', 'unknown')\n                content_preview = first_doc.page_content[:200]\n                \n                print(f'📄 Источник: {source}')\n                print(f'📝 Превью: {content_preview}...')\n                \n                # Получаем форматированный контекст\n                context = knowledge_manager.get_knowledge_context(agent_id, query, k=2)\n                \n                rag_demo_results[agent_id] = {\n                    'query': query,\n                    'documents_found': len(relevant_docs),\n                    'total_chars': total_chars,\n                    'search_time': search_time,\n                    'context_length': len(context),\n                    'success': True\n                }\n            else:\n                print('❌ Релевантные документы не найдены')\n                rag_demo_results[agent_id] = {\n                    'query': query,\n                    'documents_found': 0,\n                    'success': False\n                }\n                \n        except Exception as e:\n            print(f'❌ Ошибка поиска: {str(e)}')\n            rag_demo_results[agent_id] = {\n                'query': query,\n                'error': str(e),\n                'success': False\n            }\n    else:\n        print(f'⚠️ Агент {agent_id} не найден')\n\n# Итоговая статистика RAG поиска\nprint('\n' + '=' * 60)\nprint('📊 СТАТИСТИКА RAG ПОИСКА:')\nprint('=' * 60)\n\nsuccessful_searches = len([r for r in rag_demo_results.values() if r.get('success', False)])\ntotal_docs_found = sum(r.get('documents_found', 0) for r in rag_demo_results.values())\ntotal_chars = sum(r.get('total_chars', 0) for r in rag_demo_results.values())\navg_search_time = sum(r.get('search_time', 0) for r in rag_demo_results.values() if 'search_time' in r) / max(1, successful_searches)\n\nprint(f'✅ Успешных поисков: {successful_searches}/{len(test_queries)}')\nprint(f'📄 Всего найдено документов: {total_docs_found}')\nprint(f'📝 Всего символов контекста: {total_chars:,}')\nprint(f'⏱️ Средняя скорость поиска: {avg_search_time:.3f}с')\n\n# Сохраняем результаты\nglobals()['RAG_DEMO_RESULTS'] = rag_demo_results\n\nprint(f'\n🎯 RAG система полностью функциональна: {successful_searches >= 4}')